# Demo: Multivariable Regression on Boston Housing Data

In [1]:
import pandas as pd
import numpy as np

names =['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',  'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data',
                 header=None, delim_whitespace=True, names=names, na_values='?')

"""
Attribute Information:
    1.  CRIM      per capita crime rate by town
    2.  ZN        proportion of residential land zoned for lots over
                  25,000 sq.ft.
    3.  INDUS     proportion of non-retail business acres per town
    4.  CHAS      Charles River dummy variable (= 1 if tract bounds
                  river; 0 otherwise)
    5.  NOX       nitric oxides concentration (parts per 10 million)
    6.  RM        average number of rooms per dwelling
    7.  AGE       proportion of owner-occupied units built prior to 1940
    8.  DIS       weighted distances to five Boston employment centres
    9.  RAD       index of accessibility to radial highways
    10. TAX       full-value property-tax rate per $10,000
    11. PTRATIO   pupil-teacher ratio by town
    12. B         1000(Bk - 0.63)^2 where Bk is the proportion of blocks by town
    13. LSTAT     % lower status of the population
    14. MEDV      Median value of owner-occupied homes in $1000's
"""

"\nAttribute Information:\n    1.  CRIM      per capita crime rate by town\n    2.  ZN        proportion of residential land zoned for lots over \n                  25,000 sq.ft.\n    3.  INDUS     proportion of non-retail business acres per town\n    4.  CHAS      Charles River dummy variable (= 1 if tract bounds \n                  river; 0 otherwise)\n    5.  NOX       nitric oxides concentration (parts per 10 million)\n    6.  RM        average number of rooms per dwelling\n    7.  AGE       proportion of owner-occupied units built prior to 1940\n    8.  DIS       weighted distances to five Boston employment centres\n    9.  RAD       index of accessibility to radial highways\n    10. TAX       full-value property-tax rate per $10,000\n    11. PTRATIO   pupil-teacher ratio by town\n    12. B         1000(Bk - 0.63)^2 where Bk is the proportion of blocks by town\n    13. LSTAT     % lower status of the population\n    14. MEDV      Median value of owner-occupied homes in $1000's\n"

## Forming the Design Matrix

We want to put our features into feature vectors (stacked into a design matrix). Here we check the difference between the numpy and pandas datatype, and see the importance of using ```df['feature'].values``` to get a numpy array returned.

In [2]:
print(df.columns.to_list())
features=df.columns.to_list()
features.remove('PRICE')
print(features)

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


Treat all the features as a vector, $\mathbf{x}$, and stack the samples in a $N$ by $D$ matrix, $X$, where $N$ is the number of samples and $D$ is the number of features.

In [3]:
print(df)

        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296.0   
1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242.0   
2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242.0   
3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222.0   
4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222.0   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...    ...   
501  0.06263   0.0  11.93     0  0.573  6.593  69.1  2.4786    1  273.0   
502  0.04527   0.0  11.93     0  0.573  6.120  76.7  2.2875    1  273.0   
503  0.06076   0.0  11.93     0  0.573  6.976  91.0  2.1675    1  273.0   
504  0.10959   0.0  11.93     0  0.573  6.794  89.3  2.3889    1  273.0   
505  0.04741   0.0  11.93     0  0.573  6.030  80.8  2.5050    1  273.0   

     PTRATIO       B  LSTAT  PRICE  
0       15.3  396.90   4.98   24.0  
1       17.8  396.90   9.

In [6]:
# Features

# PRICE should not be a feature as that's what you predict. If model already knows prices, there is nothing to predict
X = df[features].values
print(X)
print(X.shape)
print(f"The dataset contains {X.shape[0]} data points with {X.shape[1]} features")

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
(506, 13)
The dataset contains 506 data points with 13 features


In [7]:
# Labels
y = df['PRICE'].values
print(y.shape)
y = y.reshape(-1,1)
print(y.shape)

(506,)
(506, 1)


# Linear Regression

We are going to use [sklearn](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares).

First, we define a linear regression model and we fit the model to our data.

[LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [23]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression(fit_intercept=True)
regr.fit(X, y)

LinearRegression()

The accuracy of the model could be evaluated by finding the MSE between the model prediction and corresponding data points

In [11]:
y_hat = regr.predict(X)  # Model prediction
print(y_hat.shape)

(506, 1)


In [12]:
mse_y = np.mean((y-y_hat)**2)
print(mse_y)

21.894831181729202


### Here are the parameter of the model :

In [13]:
print(regr.coef_)        # this is [w_1, ...., w_n]
print(regr.intercept_)   # this the bias w_0

[[-1.08011358e-01  4.64204584e-02  2.05586264e-02  2.68673382e+00
  -1.77666112e+01  3.80986521e+00  6.92224640e-04 -1.47556685e+00
   3.06049479e-01 -1.23345939e-02 -9.52747232e-01  9.31168327e-03
  -5.24758378e-01]]
[36.45948839]


### Here is a fancy way to compare $y$ and $\hat{y}$ :

In [15]:
Y = np.hstack([y, y_hat])
with np.printoptions(precision=2):
    print(Y[:10,:]) # Slicing used to get first 10 rows. But all columns

[[24.   30.  ]
 [21.6  25.03]
 [34.7  30.57]
 [33.4  28.61]
 [36.2  27.94]
 [28.7  25.26]
 [22.9  23.  ]
 [27.1  19.54]
 [16.5  11.52]
 [18.9  18.92]]


# Exercise :
Compute the Least square solution with numpy and compare your result with the one of sklearn !

In [24]:
## To do
ones_vec=np.ones((X.shape[0],1))
X_prime = np.hstack([ones_vec,X]) # using ones allows us to include bias terms within the matric for a speedup
weights=np.linalg.inv(X_prime.T@X_prime)@X_prime.T@y # Pythonic code can allow you to all steps in one line
y_hat_ml=X_prime@weights

mse_ml=np.mean((y-y_hat_ml)**2)
mae_ml=np.mean(np.abs(y-y_hat_ml))

In [25]:
print(mse_ml)

21.894831181729206


In [26]:
print(mae_ml)

3.270862810899656


You can obvserve two things


1.   Multivariate regression works to give better/lower losses as your model can take advantage of all features in dataset. A single feature cannot represent a complex dataset as well.
2.   Using existing libraries takes less time to write code for. The code is also less prone to implementation errors.

The second point explains why we use libraries so often. They have been extensively tested for edge cases and importing and using a library is less prone to small mistakes than writning it from scratch

